In [23]:
import os
import netCDF4
from netCDF4 import Dataset
import numpy as np
from matplotlib import pyplot  as plt
import time
import datetime as dt

In [35]:
''' 获取路径下全部文件名 '''
def eachFile(filepath):
    pathDir =  os.listdir(filepath)
    f = []
    for allDir in pathDir:
        child = os.path.join('%s%s' % (filepath, allDir))
        f.append(child)
#        print (child) # .decode('gbk')是解决中文显示乱码问题
    return f

''' Read date'''
def ReadDate(filename):
    file = Dataset(filename)
    # Read date
    T = file.variables['time']
    date = netCDF4.num2date(T[:], T.units, T.calendar)
    
    file.close()
    return date

''' 读nc海冰密集度 '''
def ReadSIC(filename,day):
    file = Dataset(filename)
    
    # Read sic
    sic = file.variables['siconc'][day]
    # Read lat
    lat = file.variables['latitude'][:]
    # 限定研究范围在84N~88.3N
    lim1 = np.where(lat==88.25)[0][0]
    lim2 = np.where(lat==83.75)[0][0]
    sic = sic[lim1:lim2,:]   
    
    file.close()
    return sic


def CountLow(data,Grid_Area):

        
    ''' 统计低值出现频次 '''   
    """
    Input: data,冰密集度日数据; 
           lim1,纬度下限;
           lim2,纬度上限
    Output: a_low,低值面积;
            a_total,总面积;
            LCCA,低值面积/总面积  
    """ 

#    Grid_Area = np.load('C:\\Users\\kathy\\research\\Sea-ice-concentration\\data\\ERA 5\\Geodata\\Grid_Area_125.npy')
#    Grid_Area = Grid_Area[14:48,:]  # 84N~88.3N
    ''' 统计研究范围总面积 —— 84N~90N '''
    a_total = np.sum(Grid_Area)  
#    print('总面积：%f' %a_total)  #  显示总面积
    '''统计低值面积 —— sic小于75%且大于15%'''
    a_low = np.sum(Grid_Area[data<0.75])-np.sum(Grid_Area[data<0.15])
#    print('低值面积：%f' %a_low)  #  显示低值面积
    lcca = a_low/a_total*100 
    return lcca

In [36]:
''' main 函数  '''
#start = time.clock()

# 测试路径
#FilePath = "D:\\AMSR2-NT2\\2012\\B01\\test\\"  
# 测试文件
#filename = "C:\\Users\\kathy\\research\\Sea-ice-concentration\\data\\ERA 5\\2002_2018\\era5_sic_20020601_20021031.nc"

FilePath = "C:\\Users\\kathy\\research\\Sea-ice-concentration\\data\\ERA 5\\2002_2018\\"
# FileName = eachFile(FilePath)
FileName = ['C:\\Users\\kathy\\research\\Sea-ice-concentration\\data\\ERA 5\\2002_2018\\era5_sic_20190601_20191031.nc']
print(FileName)

# =============================================================================
# Read cell area
# =============================================================================
f = Dataset('C:\\Users\\kathy\\research\\Sea-ice-concentration\\data\\ERA 5\\Geodata\\ERA_dot25_gridarea.nc')
cell_area = f.variables['cell_area'][:]
# Read lat
lat = f.variables['latitude'][:]
# 限定研究范围在84N~88.3N
lim1 = np.where(lat==88.25)[0][0]
lim2 = np.where(lat==83.75)[0][0]
cell_area = cell_area[lim1:lim2,:]
f.close()

# =============================================================================
# Year Loop
# =============================================================================

for filename in FileName:
    LowArea = []
    ts = []
    
    # Read date
    Date = ReadDate(filename)
    yr = Date[0].year
    
    file = open('ERA5-SIC-LCCA-'+str(yr)+'-KD.txt','w')      
    
    for i in range(len(Date)):
        # Read sea-ice-concentration
        SIC = ReadSIC(filename,i)
        # Calculate lcca
        lcca = CountLow(SIC,cell_area)
        # save date and value
        date = dt.datetime.strftime(Date[i],'%Y%m%d')
#         print(date)
        file.write('%10s %.2f' %(date,lcca))
        file.write('\n')
    
    file.close()
    
    print('success')

['C:\\Users\\kathy\\research\\Sea-ice-concentration\\data\\ERA 5\\2002_2018\\era5_sic_20190601_20191031.nc']
success
